In [12]:
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import torch as tc
genders = ['female', 'male']
male = ['male']
female = ['female']

In [23]:
their_semantics = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Upper arm girth R',
    'Neck girth',
    'Back neck point to waist',
    'Height (m)',
]
our_semantic = [
    'subject',
    'measures_station',
    'repetition',
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'thigh_girth', # 5.3.20
    'upper_arm_girth', # 5.3.16
    'neck_girth', # 5.3.2
    'neck_waist',
    'stature', # 5.1.1
]

station_list = ["MOVE4D", "Expert", "Our"]

measures = pd.read_pickle(f'data/cleaned_measures.zip')
measures = measures[their_semantics]
measures.columns = our_semantic
our_measures = pd.read_pickle("data/our_measures.zip")
measures = pd.concat([our_measures, measures]).drop(columns=["gender", "stature"])
measures = measures[measures['measures_station'].isin(["MOVE4D", "Expert measurer 1", "our"])]
measures.replace("Expert measurer 1", "Expert", inplace=True)
measures.replace("our", "Our", inplace=True)

In [24]:
measures

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist,repetition,measures_station,subject
0,879.825195,750.751343,1053.165405,626.480164,297.009918,329.085876,411.729614,1,Our,IEEEP2_01
1,883.476624,752.710327,1055.366943,623.614441,298.111176,325.854523,419.242859,2,Our,IEEEP2_01
2,896.443237,682.851685,948.505310,535.620667,276.295074,316.661316,412.113953,1,Our,IEEEP2_03
3,882.155090,665.351685,952.023193,535.229431,276.825439,312.521088,406.582611,2,Our,IEEEP2_03
4,1098.579834,980.695679,1137.411987,669.249634,347.686432,372.356293,431.701660,1,Our,IEEEP2_07
...,...,...,...,...,...,...,...,...,...,...
1992,1190.000000,1120.000000,1360.000000,740.000000,380.000000,375.000000,405.000000,2,Expert,IEEEP2_71
1997,931.730000,864.896000,988.994000,572.149000,319.426000,370.596000,442.053000,1,MOVE4D,IEEEP2_72
1998,922.700000,863.313000,993.492000,572.987000,320.638000,369.121000,439.612000,2,MOVE4D,IEEEP2_72
2007,930.000000,825.000000,985.000000,565.000000,305.000000,355.000000,435.000000,1,Expert,IEEEP2_72


# CALC ICC

In [25]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure)['ICC'].loc[2]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [26]:
a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.972840,72.638690,71,72,9.770264e-48,"[0.96, 0.98]"
1,ICC2,Single random raters,0.972836,71.836556,71,71,5.396240e-47,"[0.96, 0.98]"
2,ICC3,Single fixed raters,0.972541,71.836556,71,71,5.396240e-47,"[0.96, 0.98]"
3,ICC1k,Average raters absolute,0.986233,72.638690,71,72,9.770264e-48,"[0.98, 0.99]"
4,ICC2k,Average random raters,0.986231,71.836556,71,71,5.396240e-47,"[0.98, 0.99]"
5,ICC3k,Average fixed raters,0.986080,71.836556,71,71,5.396240e-47,"[0.98, 0.99]"


In [27]:
our_icc.index = ['Expert', "MOVE4D", 'Our']
our_icc

,bust_chest_girth,waist_girth,hip_girth,thigh_girth,upper_arm_girth,neck_girth,neck_waist
Expert,0.989452,0.989987,0.987519,0.980579,0.940527,0.979811,0.910240
MOVE4D,0.994299,0.997657,0.999204,0.998120,0.995792,0.995974,0.973715
Our,0.997225,0.998157,0.998845,0.998282,0.993855,0.994873,0.972541


# CALC SEM

In [28]:
all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in our_semantic[3:-1]:
        a = measures[measures['measures_station'] == station][['subject', 'repetition', measure]]
        icc = pg.intraclass_corr(data=a, targets='subject', raters='repetition', ratings=measure, nan_policy='omit')['ICC'].loc[2]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, our_semantic[3:-1]).sort_index()

In [29]:
our_sem.index = ['Expert', "MOVE4D", 'Our']
our_sem.T.round(4)

,Expert,MOVE4D,Our
bust_chest_girth,11.7387,7.9139,5.4395
waist_girth,13.1368,6.4644,5.7862
hip_girth,11.6244,2.7550,3.1412
thigh_girth,8.9775,2.5773,2.5971
upper_arm_girth,10.0674,2.4972,2.7519
neck_girth,5.3662,2.3456,2.6580
neck_waist,9.9700,4.8049,3.9726


# CALC PSD

In [30]:
def calculate_psd(measure):
    data = measures
    f1 = data['repetition'] == 1
    f2 = data['repetition'] == 2
    data = data[ f1 | f2 ][['subject', 'measures_station', 'repetition', measure]]
    data = data.pivot_table(values=measure, index=['subject', 'repetition'], columns="measures_station")
    data = data.dropna()
    

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [32]:
neck_result = calculate_psd(measure='neck_girth')
MX = pd.Series({"MOVE/Expert": neck_result.loc["MOVE4D"]["Expert"]})
neck_result = neck_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/342016044.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neck_result = neck_result.loc['Our'][:-1].append(MX)


In [35]:
bust_result = calculate_psd(measure='bust_chest_girth')
MX = pd.Series({"MOVE/Expert": bust_result.loc["MOVE4D"]["Expert"]})
bust_result = bust_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/3277463622.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bust_result = bust_result.loc['Our'][:-1].append(MX)


In [38]:
waist_result = calculate_psd(measure='waist_girth')
MX = pd.Series({"MOVE/Expert": waist_result.loc["MOVE4D"]["Expert"]})
waist_result = waist_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/2919692699.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  waist_result = waist_result.loc['Our'][:-1].append(MX)


In [39]:
hip_result = calculate_psd(measure='hip_girth')
MX = pd.Series({"MOVE/Expert": hip_result.loc["MOVE4D"]["Expert"]})
hip_result = hip_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/3292477041.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hip_result = hip_result.loc['Our'][:-1].append(MX)


In [40]:
arm_result = calculate_psd(measure='upper_arm_girth')
MX = pd.Series({"MOVE/Expert": arm_result.loc["MOVE4D"]["Expert"]})
arm_result = arm_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/4200355893.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  arm_result = arm_result.loc['Our'][:-1].append(MX)


In [58]:
thigh_result = calculate_psd(measure='thigh_girth')
MX = pd.Series({"MOVE/Expert": thigh_result.loc["MOVE4D"]["Expert"]})
thigh_result = thigh_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/3944538916.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  thigh_result = thigh_result.loc['Our'][:-1].append(MX)


In [59]:
neck_waist_result = calculate_psd(measure='neck_waist')
MX = pd.Series({"MOVE/Expert": neck_waist_result.loc["MOVE4D"]["Expert"]})
neck_waist_result = neck_waist_result.loc['Our'][:-1].append(MX)

/tmp/ipykernel_701367/4146834841.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neck_waist_result = neck_waist_result.loc['Our'][:-1].append(MX)


In [64]:
def calculate_mad():
    all_measures = []
    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        f1 = a['repetition'] == 1
        f2 = a['repetition'] == 2
        for measure in our_semantic[3:-1]:
            mean = np.nanmean(abs(a[f1][measure].to_numpy() - a[f2][measure].to_numpy()))
            measures_of_station.append(mean)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=['MOVE4D','Expert','Our']).T

In [72]:
calculate_mad().round(4).to_csv("output/our_mad.csv")
calculate_mad().round(4)

,MOVE4D,Expert,Our
bust_chest_girth,8.5292,11.5157,6.5871
waist_girth,6.4874,12.2047,5.6416
hip_girth,3.0823,10.8465,3.6243
thigh_girth,2.6278,9.9016,2.8898
upper_arm_girth,2.7248,6.8150,2.7044
neck_girth,2.3442,5.8071,2.8955
neck_waist,5.1012,10.9646,4.5123


In [79]:
our_sem.T.round(4)

,Expert,MOVE4D,Our
bust_chest_girth,11.7387,7.9139,5.4395
waist_girth,13.1368,6.4644,5.7862
hip_girth,11.6244,2.7550,3.1412
thigh_girth,8.9775,2.5773,2.5971
upper_arm_girth,10.0674,2.4972,2.7519
neck_girth,5.3662,2.3456,2.6580
neck_waist,9.9700,4.8049,3.9726


In [80]:
data_psd = pd.DataFrame(
    [neck_result, bust_result, waist_result, hip_result, arm_result,thigh_result, neck_waist_result],
    index=our_semantic[3:-1]).round(2)
data_psd.columns = ['Our vs Expert', "Our vs MOVE4D", 'MOVE vs Expert']
data_psd.to_csv("output/our_psd.csv")
data_psd

,Our vs Expert,Our vs MOVE4D,MOVE vs Expert
bust_chest_girth,11.96,6.85,10.31
waist_girth,23.70,21.11,22.27
hip_girth,24.34,13.51,23.30
thigh_girth,16.73,13.14,15.96
upper_arm_girth,16.04,6.42,14.79
neck_girth,14.96,10.23,12.15
neck_waist,20.37,13.72,20.35


In [67]:
def calculate_msd():

    all_measures = []

    for station in station_list:
        measures_of_station = []
        a = measures[measures['measures_station'] == station]
        for measure in our_semantic[3:-1]:
            measured = a[measure].to_numpy()
            ground_thruth = grand_mean[measure]
            msd = np.nanmean(measured - ground_thruth)
            measures_of_station.append(msd)
        all_measures.append(measures_of_station)
    return pd.DataFrame(all_measures, columns=our_semantic[3:-1], index=station_list).T
calculate_msd().round(4).to_csv("output/our_msd.csv")
calculate_msd().round(4)

,MOVE4D,Expert,Our
bust_chest_girth,14.9436,-18.0816,16.9504
waist_girth,11.8618,-14.4606,13.6451
hip_girth,5.7136,-4.8497,2.8408
thigh_girth,3.2407,-2.4446,1.0713
upper_arm_girth,5.2535,-6.7143,6.5899
neck_girth,0.0766,-2.9098,5.0560
neck_waist,5.4944,-6.8083,6.5146
